<img src="https://github.com/OasisLMF/Workshop2019/raw/master/images/oasis-lmf-colour.png" alt="Oasis LMF logo" width="250" align="left"/>
<br><br><br>

Exercise 4: User Access Management 
==================================
This exercise is a short introduction on how restricted access to models, portfolios and analyses via [KeyCloak](https://www.keycloak.org/documentation), an open-source identity and access management tool.
This is the default user management tool for the Oasis Enterprise Platform. 

In [ ]:
# Edit this value to match your workshop ID, if your username is `workshop6@oasislmfenterprise.onmicrosoft.com` set   'WORKSHOP_ID=6'
WORKSHOP_ID=


# Install of workshop package 
!pip install oasis-workshop==1.0.0
    
# Clear cell output
from IPython.display import clear_output
clear_output(wait=False) 

In [ ]:
from oasis_workshop.client import APIClient
from oasis_workshop.funcs import (
    tabulate_endpoint, 
    tabulate_json,
    tabulate_analysis,
    tabulate_portfolio
)
    
from IPython.display import (
    display, 
    Code,
    clear_output, 
    HTML, 
    JSON,
    Markdown 
)

display(Markdown(f'# Keycloak URL - https://oasis-workshop-{WORKSHOP_ID}.northcentralus.cloudapp.azure.com/auth/'))

## 4.1 KeyCloak setup 

### 4.1.1 Import groups and users to KeyCloak 
Open the URL above and login using these credentials. 

```
username: keycloak
password: password
```


#### 1. Save the example user and groups data 
Download a local copy of the file [
users-of-middle-earth.json](https://raw.githubusercontent.com/OasisLMF/Workshop2022/main/examples/users-of-middle-earth.json)


#### 2. Go to the **import** page in Keycloak
From there you can select the saved `users-of-middle-earth.json` file and load it.

<img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/import_users_1.png?raw=true" alt="chunking" width="600" align="center" style="float"/>

#### 3. Import and Override 
Once the file has loaded the page should look like the screenshot below, select `overwrite` from the dropdown and hit import.

<img src="https://github.com/OasisLMF/Workshop2022/blob/main/images/import_users_2.png?raw=true" alt="chunking" width="600" align="center" style="float"/>




### 4.1.2 Start user sessions 

In [ ]:
server_url = f'https://oasis-workshop-{WORKSHOP_ID}.northcentralus.cloudapp.azure.com/api/'
display_cols = ['id', 'supplier_id', 'model_id', 'version_id', 'groups']

# Start API connections
session_iluvatar = APIClient(api_url=server_url, username='iluvatar', password='pass')
session_faramir = APIClient(api_url=server_url, username='faramir', password='pass')
session_frodo = APIClient(api_url=server_url, username='frodo', password='pass')
session_gandalf = APIClient(api_url=server_url, username='gandalf', password='notpass')
session_gollum = APIClient(api_url=server_url, username='gollum', password='pass')
session_sauron = APIClient(api_url=server_url, username='sauron', password='pass')

## 4.2 Admin account see all Objects.
From the connected sessions only the account `iluvatar` is admin, the others are locked into set groups which were imported along with the user data.


| **Username** | **Groups** | **is_admin** |
|:-------------|------------|--------------|
| iluvatar     | admin, middle_earth | True     |
| faramir      | gondor              | False    |
| frodo        | the_shire           | False    |
| gandalf      | gondor, the_shire   | False    |
| gollum       | -None-              | False    |
| sauron       | mordor              | False    |



In [ ]:
display(Markdown(f"### Iluvatar's CAT Models"))
display(Markdown(f"Note that the `groups` column is empty, thats because these were created by an admin account from ex2"))
display(HTML(tabulate_endpoint(session_iluvatar.models, display_cols)))

display(Markdown(f"### All the non-admin users return empty lists"))
display(Markdown(f"The same applies when logging into the OasisUI with these accounts"))
display(Markdown(f" * faramir - {session_faramir.models.get().json()}"))
display(Markdown(f" * frodo - {session_frodo.models.get().json()}"))
display(Markdown(f" * gandalf - {session_gandalf.models.get().json()}"))
display(Markdown(f" * sauron - {session_sauron.models.get().json()}"))

## 4.3 User created Objects inherit group permissions
This applies to `models` and `portfolios`

In [ ]:
if not session_faramir.models.search({'supplier_id': 'minas_tirith'}).json():
    session_faramir.models.create('minas_tirith', 'orc_interruption', '6.0')
if not session_frodo.models.search({'supplier_id': 'mordor'}).json():
    session_frodo.models.create('mordor', 'travel_risk', '0.01')
if not session_frodo.models.search({'supplier_id': 'hobbiton'}).json():
    session_frodo.models.create('hobbiton', 'horticulture', '3')
if not session_gandalf.models.search({'supplier_id': 'rohan'}).json():
    session_gandalf.models.create('rohan', 'support_response_time', '2')
if not session_sauron.models.search({'supplier_id': 'mount_doom'}).json():
    session_sauron.models.create('mount_doom', 'eruption', '1.0')
if not session_sauron.models.search({'supplier_id': 'one_ring'}).json():
    session_sauron.models.create('one_ring', 'detection', '1.0')
    
# Patch orig groups
rohan_model = session_gandalf.models.search({'supplier_id': 'rohan'}).json().pop()
session_gandalf.models.patch(rohan_model['id'], {'groups': ['gondor', 'the_shire']})

In [ ]:
display(Markdown(f"### Faramir's Models"))
display(Markdown(f"Note that the `groups` column is empty, thats because these were created by an admin account from ex2"))
display(HTML(tabulate_endpoint(session_faramir.models, display_cols)))

display(Markdown(f"### Frodo's CAT Models"))
display(Markdown(f"Note that the `groups` column is empty, thats because these were created by an admin account from ex2"))
display(HTML(tabulate_endpoint(session_frodo.models, display_cols)))

display(Markdown(f"### Gandalf's CAT Models"))
display(Markdown(f"Note that the `groups` column is empty, thats because these were created by an admin account from ex2"))
display(HTML(tabulate_endpoint(session_gandalf.models, display_cols)))

display(Markdown(f"### Sauron's CAT Models"))
display(Markdown(f"Note that the `groups` column is empty, thats because these were created by an admin account from ex2"))
display(HTML(tabulate_endpoint(session_sauron.models, display_cols)))

display(Markdown(f"### Iluvatar's CAT Models"))
display(Markdown(f"Note that the `groups` column is empty, thats because these were created by an admin account from ex2"))
display(HTML(tabulate_endpoint(session_iluvatar.models, display_cols)))

### 4.3.1 Empty groups are treated as a group

Like in set theory, an [empty set](https://en.wikipedia.org/wiki/Empty_set), or group, is still treated as a set. 
If a user belongs to a group it won't be able to access objects that belong to a group and vice versa.
Since `gollum` isn't a member of a group, the two models from ex2 are accessible. 

In [ ]:
display(Markdown(f"### Gollum's CAT Models"))
display(Markdown(f"Note that the `groups` column is empty, thats because these were created by an admin account from ex2"))
display(HTML(tabulate_endpoint(session_gollum.models, display_cols)))

### 4.2.3 Analysis inherit groups from portfolios

Analysis will inherit the groups from the portfolio, but for a user to run the analysis it requires the user to be a member of at least one of the model's groups.

In [ ]:
if not session_gandalf.portfolios.search({'name': 'lit_beacons'}).json():
    PORT_ID = session_gandalf.portfolios.create('lit_beacons').json()['id']
else:
    PORT_ID = session_gandalf.portfolios.search({'name': 'lit_beacons'}).json().pop()['id']

if not session_gandalf.analyses.search({'name': 'gondor_call_for_aid'}).json():
    session_gandalf.portfolios.location_file.post(PORT_ID, '-n/a-', content_type='text/csv')
    MODEL_ID = session_gandalf.models.search({'supplier_id': 'rohan'}).json().pop()['id']
    session_gandalf.analyses.create('gondor_call_for_aid', PORT_ID, MODEL_ID) 

display(Markdown(f"### Gandalf's Portfolios"))
display(HTML(tabulate_endpoint(session_gandalf.portfolios, ['id', 'name', 'groups'])))


display(Markdown(f"### Gandalf's Analyses"))
display(HTML(tabulate_endpoint(session_gandalf.analyses, ['id', 'name', 'model', 'portfolio', 'status', 'groups'])))

### 4.4 Removing access to a model. 

Given a change in travel plans, Frodo no longer needs access to the `rohan, support_response_time` model. 
Gandalf can edit the `groups` field and remove **the_shire** field


In [ ]:
rohan_model = session_gandalf.models.search({'supplier_id': 'rohan'}).json().pop()
session_gandalf.models.patch(rohan_model['id'], {'groups': ['gondor']})

display(Markdown(f"### Frodo's CAT Models"))
display(Markdown(f"Frodo's access to the rohan model was removed"))
display(HTML(tabulate_endpoint(session_frodo.models, display_cols)))


display(Markdown(f"### Frodo's Analyses"))
display(Markdown(f"However can still see the analyses created before the model group removal"))
display(HTML(tabulate_endpoint(session_frodo.analyses,  ['id', 'name', 'model', 'portfolio', 'status', 'groups'])))

### 4.4.1 Users to both Models and Analyses to execute 

In [ ]:
ANALYSIS_ID = session_frodo.analyses.search({'name':'gondor_call_for_aid'}).json().pop()['id']

try:
    r = session_frodo.analyses.generate(ANALYSIS_ID)
except Exception as e: 
    print(f'HTTP Error: {e.args[1].response.status_code}')
    print(f'Msg: {e.args[1].response.text}')

### 4.4.2 Users can only modify objects which have the same group.

The user `sauron` wants to add **rohan** to the **mordor** group, but even with the ID the API returns 403

In [ ]:
rohan_model = session_gandalf.models.search({'supplier_id': 'rohan'}).json().pop()
r = session_sauron.models.patch(rohan_model['id'], {'groups': ['mordor']})

print(f'HTTP Error: {r.status_code}')
print(f'Msg: {r.text}')